# Debris Pile Detection for Hurricane Relief
## Pinellas County, FL - Post Hurricane Milton/Helene

This notebook uses **SamGeo** (Segment Anything Model for Geospatial Data) to automatically detect debris piles from satellite imagery. Results can help Red Cross target relief efforts.

**Data Sources:**
- NOAA NGS Aerial Imagery: https://storms.ngs.noaa.gov/storms/milton/index.html
- Maxar Open Data: s3://maxar-opendata/events/HurricaneMilton-Oct24/

## 1. Install Dependencies

Run this cell first to install required packages. **Requires GPU for best performance.**

In [ ]:
# Install SamGeo with text prompt support
# Uncomment the line below if running in Google Colab or fresh environment
# !pip install segment-geospatial[text] leafmap geopandas folium

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Import Libraries

In [ ]:
import leafmap
from samgeo import SamGeo, tms_to_geotiff
from samgeo.text_sam import LangSAM
import geopandas as gpd
from pathlib import Path
import os

# Create output directory
OUTPUT_DIR = Path("./output")
OUTPUT_DIR.mkdir(exist_ok=True)
print(f"Output directory: {OUTPUT_DIR.absolute()}")

## 3. Define Pinellas County Areas of Interest

These are key areas in Pinellas County that were affected by Hurricane Milton. Adjust the bounding boxes as needed.

In [ ]:
# Pinellas County areas - [west, south, east, north] in WGS84
PINELLAS_AREAS = {
    "clearwater_beach": [-82.835, 27.965, -82.815, 27.985],
    "st_pete_beach": [-82.745, 27.715, -82.725, 27.735],
    "treasure_island": [-82.780, 27.760, -82.760, 27.780],
    "indian_rocks_beach": [-82.860, 27.880, -82.840, 27.900],
    "dunedin": [-82.795, 28.010, -82.775, 28.030],
    "largo": [-82.795, 27.900, -82.775, 27.920],
    "pinellas_park": [-82.715, 27.835, -82.695, 27.855],
    "gulfport": [-82.715, 27.740, -82.695, 27.760],
    "madeira_beach": [-82.810, 27.790, -82.790, 27.810],
}

# Select area to process
SELECTED_AREA = "clearwater_beach"  # Change this to process different areas
bbox = PINELLAS_AREAS[SELECTED_AREA]
print(f"Selected area: {SELECTED_AREA}")
print(f"Bounding box: {bbox}")

## 4. Create Interactive Map to Select Area

In [ ]:
# Create interactive map centered on Pinellas County
m = leafmap.Map(center=[27.85, -82.75], zoom=11)

# Add satellite imagery layer
m.add_basemap("SATELLITE")

# Add bounding boxes for all defined areas
for name, box in PINELLAS_AREAS.items():
    m.add_bbox(box, layer_name=name)

m

## 5. Download Satellite Imagery

Download high-resolution satellite imagery for the selected area.

In [ ]:
# Output path for imagery
image_path = OUTPUT_DIR / f"{SELECTED_AREA}_satellite.tif"

# Download satellite imagery (using ESRI World Imagery)
# For NOAA post-hurricane imagery, you'll need to download from their portal
print(f"Downloading satellite imagery for {SELECTED_AREA}...")
print(f"Bounding box: {bbox}")

tms_to_geotiff(
    output=str(image_path),
    bbox=bbox,
    zoom=18,  # High resolution for debris detection
    source="Satellite",
    quiet=False
)

print(f"\nImagery saved to: {image_path}")

## 6. View Downloaded Imagery

In [ ]:
# Display the downloaded imagery on map
m2 = leafmap.Map()
m2.add_raster(str(image_path), layer_name="Satellite Imagery")
m2

## 7. Initialize LangSAM for Text-Based Detection

LangSAM combines SAM with language understanding to detect objects based on text descriptions.

In [ ]:
# Initialize LangSAM (this may take a moment to download models)
print("Initializing LangSAM model...")
sam = LangSAM()
print("Model ready!")

## 8. Detect Debris Piles Using Text Prompts

We'll use various text prompts to detect debris piles. The model will find objects matching these descriptions.

In [ ]:
# Define text prompts for debris detection
# Try different prompts to see what works best for your imagery
DEBRIS_PROMPTS = [
    "debris pile",
    "storm debris",
    "construction debris",
    "rubble",
    "waste pile",
    "damaged materials",
    "trash pile",
]

# Detection parameters - adjust these for better results
BOX_THRESHOLD = 0.24  # Lower = more detections, higher = fewer but more confident
TEXT_THRESHOLD = 0.24

In [ ]:
# Run detection with primary prompt
primary_prompt = "debris pile"
print(f"Detecting: '{primary_prompt}'")

sam.predict(
    image=str(image_path),
    text_prompt=primary_prompt,
    box_threshold=BOX_THRESHOLD,
    text_threshold=TEXT_THRESHOLD,
)

# Save masks as raster
mask_path = OUTPUT_DIR / f"{SELECTED_AREA}_debris_mask.tif"
sam.save_masks(
    output=str(mask_path),
    dtype="uint8",
    unique=True
)
print(f"Mask saved to: {mask_path}")

In [ ]:
# Convert to vector format (GeoJSON) for GIS use
vector_path = OUTPUT_DIR / f"{SELECTED_AREA}_debris.geojson"

sam.raster_to_vector(
    str(mask_path),
    str(vector_path)
)
print(f"Vector file saved to: {vector_path}")

# Load and display stats
gdf = gpd.read_file(vector_path)
print(f"\nDetected {len(gdf)} potential debris piles")

## 9. Visualize Detection Results

In [ ]:
# Show annotated image with detections
sam.show_anns(
    cmap="Reds",
    add_boxes=True,
    alpha=0.5,
    title=f"Debris Detection: {SELECTED_AREA}"
)

In [ ]:
# Create interactive map with results
m3 = leafmap.Map()

# Add satellite imagery
m3.add_raster(str(image_path), layer_name="Satellite")

# Add detected debris polygons
if vector_path.exists():
    m3.add_vector(
        str(vector_path),
        layer_name="Detected Debris",
        style={"color": "red", "fillColor": "red", "fillOpacity": 0.5, "weight": 2}
    )

m3

## 10. Run Detection for Multiple Prompts

Try different prompts to capture various types of debris.

In [ ]:
# Run detection for all prompts and combine results
all_results = []

for prompt in DEBRIS_PROMPTS:
    print(f"Detecting: '{prompt}'...")
    try:
        sam.predict(
            image=str(image_path),
            text_prompt=prompt,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD,
        )
        
        # Save individual results
        prompt_mask = OUTPUT_DIR / f"{SELECTED_AREA}_{prompt.replace(' ', '_')}_mask.tif"
        prompt_vector = OUTPUT_DIR / f"{SELECTED_AREA}_{prompt.replace(' ', '_')}.geojson"
        
        sam.save_masks(output=str(prompt_mask), dtype="uint8")
        sam.raster_to_vector(str(prompt_mask), str(prompt_vector))
        
        # Load results
        gdf = gpd.read_file(prompt_vector)
        gdf['prompt'] = prompt
        all_results.append(gdf)
        print(f"  Found {len(gdf)} detections")
        
    except Exception as e:
        print(f"  No detections for '{prompt}': {e}")

# Combine all results
if all_results:
    combined = gpd.pd.concat(all_results, ignore_index=True)
    combined_path = OUTPUT_DIR / f"{SELECTED_AREA}_all_debris.geojson"
    combined.to_file(combined_path, driver="GeoJSON")
    print(f"\nTotal detections: {len(combined)}")
    print(f"Combined results saved to: {combined_path}")

## 11. Process Multiple Areas (Batch Processing)

In [ ]:
def process_area(area_name, bbox, prompts=["debris pile"]):
    """
    Process a single area for debris detection.
    
    Returns:
        GeoDataFrame with detected debris polygons
    """
    print(f"\n{'='*50}")
    print(f"Processing: {area_name}")
    print(f"{'='*50}")
    
    # Download imagery
    img_path = OUTPUT_DIR / f"{area_name}_satellite.tif"
    if not img_path.exists():
        tms_to_geotiff(output=str(img_path), bbox=bbox, zoom=18, source="Satellite", quiet=True)
    
    # Run detection
    results = []
    for prompt in prompts:
        try:
            sam.predict(image=str(img_path), text_prompt=prompt, box_threshold=0.24, text_threshold=0.24)
            mask_path = OUTPUT_DIR / f"{area_name}_{prompt.replace(' ', '_')}_mask.tif"
            vec_path = OUTPUT_DIR / f"{area_name}_{prompt.replace(' ', '_')}.geojson"
            sam.save_masks(output=str(mask_path), dtype="uint8")
            sam.raster_to_vector(str(mask_path), str(vec_path))
            gdf = gpd.read_file(vec_path)
            gdf['area'] = area_name
            gdf['prompt'] = prompt
            results.append(gdf)
        except:
            pass
    
    if results:
        return gpd.pd.concat(results, ignore_index=True)
    return None

# Process first 3 areas as example (uncomment to run)
# all_debris = []
# for name, bbox in list(PINELLAS_AREAS.items())[:3]:
#     result = process_area(name, bbox, ["debris pile", "storm debris"])
#     if result is not None:
#         all_debris.append(result)

## 12. Export Results for Red Cross

Export detection results in formats useful for relief coordination.

In [ ]:
# Export to various formats
if vector_path.exists():
    gdf = gpd.read_file(vector_path)
    
    # Add centroid coordinates for easy reference
    gdf['centroid_lon'] = gdf.geometry.centroid.x
    gdf['centroid_lat'] = gdf.geometry.centroid.y
    gdf['area_sqm'] = gdf.geometry.area
    
    # Export as GeoJSON (for web maps)
    gdf.to_file(OUTPUT_DIR / f"{SELECTED_AREA}_debris_final.geojson", driver="GeoJSON")
    
    # Export as Shapefile (for ArcGIS/QGIS)
    gdf.to_file(OUTPUT_DIR / f"{SELECTED_AREA}_debris.shp")
    
    # Export as CSV (for spreadsheet analysis)
    csv_data = gdf.drop(columns=['geometry']).copy()
    csv_data.to_csv(OUTPUT_DIR / f"{SELECTED_AREA}_debris.csv", index=False)
    
    print("Exported files:")
    print(f"  - GeoJSON: {SELECTED_AREA}_debris_final.geojson")
    print(f"  - Shapefile: {SELECTED_AREA}_debris.shp")
    print(f"  - CSV: {SELECTED_AREA}_debris.csv")
    print(f"\nTotal debris piles detected: {len(gdf)}")

## 13. Create Summary Report

In [ ]:
# Generate summary statistics
if vector_path.exists():
    gdf = gpd.read_file(vector_path)
    
    print("\n" + "="*60)
    print("DEBRIS DETECTION SUMMARY REPORT")
    print(f"Area: {SELECTED_AREA.replace('_', ' ').title()}")
    print("="*60)
    print(f"\nTotal Debris Piles Detected: {len(gdf)}")
    print(f"\nBounding Box (EPSG:4326):")
    print(f"  West:  {bbox[0]:.4f}")
    print(f"  South: {bbox[1]:.4f}")
    print(f"  East:  {bbox[2]:.4f}")
    print(f"  North: {bbox[3]:.4f}")
    print(f"\nOutput Files:")
    for f in OUTPUT_DIR.glob(f"{SELECTED_AREA}*"):
        print(f"  - {f.name}")
    print("\n" + "="*60)
    print("Ready for Red Cross relief coordination")
    print("="*60)

---

## Tips for Better Detection

1. **Use NOAA post-hurricane imagery** for best results - the current satellite imagery may not show post-hurricane conditions
2. **Adjust thresholds**: Lower `BOX_THRESHOLD` and `TEXT_THRESHOLD` to detect more objects (may include false positives)
3. **Try different prompts**: "construction debris", "rubble pile", "damaged building materials", "yard waste"
4. **Increase zoom level** for higher resolution imagery (but slower download)
5. **Process smaller areas** for faster results

## NOAA Imagery Access

For actual post-hurricane imagery:
- **Web viewer**: https://storms.ngs.noaa.gov/storms/milton/index.html
- **AWS S3**: `s3://maxar-opendata/events/HurricaneMilton-Oct24/`